In [ ]:
%pip install -Uq upgini catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.6/446.6 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 26.9 MB/s eta 0:00:00
     ━━━

Prepare input data


In [ ]:
from os.path import exists
import pandas as pd

df_path = "train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)
df = df.sample(n=19_000, random_state=0)
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)

df["date"] = pd.to_datetime(df["date"])

df.sort_values("date",inplace=True)
df.reset_index(inplace=True,drop=True)
df.head()







,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [ ]:
train = df[df["date"] < "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]

In [ ]:
train_features = train.drop(columns=["sales"])
train_target = train["sales"]
test_features = test.drop(columns=["sales"])
test_target = test["sales"]

Enrich Features

In [ ]:
!pip install upgini

In [ ]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys = {
        "date": SearchKey.DATE,
    },
    cv = CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features,test_target)])

[============================================================] 100% Finished

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

f_weather_date_weather_umap_48_b39cd0c4,0.0306,100.0000,"4.9283, 5.5052, 5.6754",Upgini,Weather & climate normals data,Daily
f_weather_date_weather_umap_31_fa6d9a99,0.0257,100.0000,"5.5919, 5.0304, 4.9374",Upgini,Weather & climate normals data,Daily
f_weather_date_weather_umap_47_5123ef0a,0.0128,100.0000,"5.3281, 4.8577, 5.4067",Upgini,Weather & climate normals data,Daily
f_autofe_mul_34d11bc4,0.0100,100.0000,"7.2793, 2.6852, 2.9362",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_478be4f3,0.0095,100.0000,"-1.0889, -0.4277, 1.2567",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_50a5dcaf,0.0088,100.0000,"-0.9662, -1.1251, 0.8839",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_weather_date_weather_pca_0_d7e0a1fc,0.0087,100.0000,"27.7845, -6.8526, -12.4841",Upgini,Weather & climate normals data,Daily
f_events_date_year_cos1_9014a856,0.0078,100.0000,"0.8939, 0.8786, -0.9991",Upgini,Calendar data,Daily
f_autofe_div_89c56a5f,0.0077,100.0000,"0.3751, -0.0922, -0.1571",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_weather_date_weather_umap_34_c3ef5b4f,0.0066,100.0000,"4.7708, 5.2742, 5.6295",Upgini,Weather & climate normals data,Daily
f_autofe_div_cf5dd7be,0.0061,100.0000,"-1.0077, -0.8554, -0.3592",Upgini,"AutoFE: features from Calendar data,Markets data",Daily


Upgini,Weather & climate normals data,0.0874,14
Upgini,"AutoFE: features from Calendar data,Markets data",0.0782,37
Upgini,AutoFE: features from Markets data,0.0162,17
Upgini,AutoFE: features from Calendar data,0.0113,10
Upgini,Calendar data,0.0080,3
Upgini,Markets data,0.0008,3
Upgini,World economic indicators,0.0007,4


"Calendar data,Markets data",f_autofe_mul_34d11bc4,f_events_date_year_cos1_9014a856,f_financial_date_silver_e4e33014,*
"Calendar data,Markets data",f_autofe_mul_478be4f3,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_7d_to_7d_1y_shift_a5c3c07f,*
"Calendar data,Markets data",f_autofe_div_50a5dcaf,f_events_date_week_cos3_7525fe31,f_financial_date_silver_7d_to_7d_1y_shift_55fa8001,/
"Calendar data,Markets data",f_autofe_div_89c56a5f,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_7d_to_7d_1y_shift_a5c3c07f,/
"Calendar data,Markets data",f_autofe_div_cf5dd7be,f_events_date_week_sin1_847b5db1,f_financial_date_vix_7d_to_1y_634c77eb,/
"Calendar data,Markets data",f_autofe_div_a66c2b64,f_events_date_week_sin1_847b5db1,f_financial_date_snp500_fdb64447,/
Markets data,f_autofe_div_0dcebab1,f_financial_date_nasdaq_7d_to_1y_b00bfaa7,f_financial_date_snp500_7d_to_1y_378b9f99,/
"Calendar data,Markets data",f_autofe_div_3e22df83,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_1f195998,/
"Calendar data,Markets data",f_autofe_mul_08e89348,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_1f195998,*
Calendar data,f_autofe_mul_5feff686,f_events_date_year_cos1_9014a856,f_events_date_year_cos2_7acbaf1b,*
Calendar data,f_autofe_div_b97cf188,f_events_date_week_cos3_7525fe31,f_events_date_week_sin2_e28440a5,/



Examples of outliers with maximum value of target:
84    205
47    196
38    187
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Calculating accuracy uplift after enrichment...

which makes metrics between the train and eval_set incomparable.


Train,15213,50.3977,301.4672,184.7644,116.7028
Eval 1,3787,59.2424,486.0393,344.7526,141.2867


In [ ]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

enricher.calculate_metrics(
    train_features, train_target,
    eval_set=[(test_features, test_target)],
    estimator = model,
    scoring = "mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
-
which makes metrics between the train and eval_set incomparable.


,Dataset type,Rows,Mean target,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
0,Train,15213,50.3977,0.252850,0.161323,0.091527
1,Eval 1,3787,59.2424,0.264187,0.189178,0.075009


In [ ]:
print("Shape of train_features before transform:", train_features.shape)  # Check shape before transform
enriched_train_features = enricher.transform(train_features.copy(), keep_input=True) # Create a copy of the DataFrame to avoid modifying the original
print("enriched_train_features:", enriched_train_features)  # Check the output of transform
enriched_test_features = enricher.transform(test_features, keep_input=True)
print("Shape of enriched_test_features:", enriched_test_features.shape)  # Check shape after transform
if enriched_train_features is not None: # Check if enriched_train_features is not None before calling head()
    enriched_train_features.head()
else:
    print("enriched_train_features is None. Investigate the enricher.transform() method or the train_features DataFrame.")


Shape of train_features before transform: (15213, 3)


[============================================================] 100% Finished

You are trying to launch enrichment for 15213 rows, which will exceed the rest limit 10000.


Button(button_style='danger', description='Request a quote', style=ButtonStyle())

enriched_train_features: None


[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 0 rows.

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history



Column name,Status,Errors
date,All valid,-



Running search request, search_id=41e3ba11-43e1-4264-95eb-5d626285c850
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...
Shape of enriched_test_features: (3787, 91)
enriched_train_features is None. Investigate the enricher.transform() method or the train_features DataFrame.


In [ ]:
model.fit(train_features, train_target)
preds = model.predict(test_features)
eval_metric(test_target.values, preds, "SMAPE")

[37.65141857448004]